In [ ]:
import pyvista as pv
import numpy as np
pv.set_jupyter_backend('panel')

files = !ls *.ply
print('Local files:')
print(files,'\n\n')

# load mesh
file = files[0]
print(file,'\n')
mesh=pv.read(file)

# rotate it randomly
x_deg,y_deg,z_deg=360*np.random.rand(3)

#pivot=mesh.outline().center_of_mass() is this needed?
pivot = np.zeros(3)

mesh.rotate_x(x_deg,pivot,inplace=True)
mesh.rotate_y(y_deg,pivot,inplace=True)
mesh.rotate_z(z_deg,pivot,inplace=True)

pivot=mesh.outline().center_of_mass()
mesh.translate(-pivot,inplace=True)




# scale so that it fits into [-1,1]^3 region
scaling_factor = 1/max(abs(np.array(mesh.bounds)))
mesh.scale(scaling_factor,inplace=True)

# print data
print(mesh)

p = pv.Plotter()
p.add_mesh(mesh)
p.link_views()
p.view_isometric()
p.show()

In [ ]:
# the grid
N_linear_cells=50
resolution = N_linear_cells*np.ones(3,dtype=int)
grid = pv.UniformGrid()
grid.dimensions = resolution+1 # the grid points are the edges of the cells: add 1 for last edge
grid.spacing = 2/resolution # spacing of the grid
grid.origin = (-1,-1,-1) # origin of the grid is set so that the cube spans the [-1,1]^3 interval
grid.cell_data['occupancy'] = np.zeros(grid.n_cells,dtype=np.int8) # initialize occupancy to zero
print(grid)

pl = pv.Plotter()
pl.add_mesh(mesh,label='mesh')
pl.add_mesh(grid,show_edges=True,style='wireframe',color='yellow',label='grid')
pl.add_legend()
pl.show()

In [ ]:
# now we grab the cell centers
points = grid.cell_centers()

# and compute the distance form the mesh
points.compute_implicit_distance(mesh,inplace=True)

In [ ]:
# now threshold on value: what is the correct value?
# if the "activate" the voxel based on whether the surface intersects it in any point,
# then the threshold is sqrt(3)*l/2, where l is the size of the voxel
# if we activate the voxel on whether
# also check https://en.wikipedia.org/wiki/Line_drawing_algorithm for ideas

l = grid.spacing[0]
threshold = 3**(0.5)*l
mask = abs(points.point_data['implicit_distance']) < threshold


grid.cell_data['occupancy'][mask] = 1


grid

In [ ]:
#check to see if export works
array=grid.cell_data['occupancy'].reshape(resolution)
grid.cell_data['occupancy'] = array

pl = pv.Plotter(shape=(1,2))
pl.link_views()
pl.add_mesh(mesh)
pl.subplot(0,1)
pl.add_mesh(grid.threshold(0.5),color='red')
pl.view_vector([0.7,0.7,1.2])

pl.show()

In [ ]:
np.save(file.split('.')[0]+'_voxel',array)

In [ ]:
!ls